In [ ]:
# ============================================================
# 逻辑回归 (Logistic Regression)
# ============================================================
# 
# 逻辑回归是用于二分类问题的经典算法
# 虽然名字里有"回归"，但它是一个分类算法
# 
# 核心思想：
# 1. 使用线性模型计算得分: z = θᵀx
# 2. 通过 Sigmoid 函数将得分映射到概率: p = σ(z) = 1/(1+e^(-z))
# 3. 使用阈值（通常0.5）进行分类
# 
# 损失函数：交叉熵 (Cross-Entropy)
# J(θ) = -1/m Σ[y·log(p) + (1-y)·log(1-p)]
# ============================================================

from sklearn import datasets

# 加载鸢尾花数据集
iris = datasets.load_iris()

print("数据集字段:", list(iris.keys()))
print(f"\n特征名称: {iris['feature_names']}")
print(f"目标类别: {iris['target_names']}")

# ============================================================
# 准备二分类数据
# ============================================================
# 任务：根据花瓣宽度(petal width)判断是否为 Virginica 类

# 只使用花瓣宽度特征（第4个特征）
X = iris['data'][:, 3:]  # 花瓣宽度

# 创建二分类标签：Virginica类(target=2)为正类(1)，其他为负类(0)
y = (iris['target'] == 2).astype(int)

print(f"\n特征形状: {X.shape}")
print(f"正类样本数: {sum(y)} (Virginica)")
print(f"负类样本数: {len(y) - sum(y)} (非Virginica)")

In [ ]:
# ============================================================
# 训练逻辑回归模型
# ============================================================

from sklearn.linear_model import LogisticRegression

# 创建逻辑回归模型
# 默认参数使用 L2 正则化 (penalty='l2')
log_reg = LogisticRegression(random_state=42)

# 训练模型
log_reg.fit(X, y)

# 查看学习到的参数
print("模型参数:")
print("-" * 40)
print(f"  截距 (intercept): {log_reg.intercept_[0]:.4f}")
print(f"  系数 (coef):      {log_reg.coef_[0, 0]:.4f}")
print("-" * 40)
print(f"\n决策边界: 当花瓣宽度 = {-log_reg.intercept_[0]/log_reg.coef_[0,0]:.2f} cm 时，概率为 0.5")

In [ ]:
# ============================================================
# 可视化预测概率曲线
# ============================================================

import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
np.random.seed(42)

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 生成测试数据点
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)

# 预测概率
# predict_proba 返回每个类别的概率 [P(y=0), P(y=1)]
y_proba = log_reg.predict_proba(X_new)

# 绘图
plt.figure(figsize=(10, 6))

# 绘制概率曲线
plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="P(Virginica)")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="P(非Virginica)")

# 标记决策边界
decision_boundary = -log_reg.intercept_[0] / log_reg.coef_[0, 0]
plt.axvline(x=decision_boundary, color='r', linestyle=':', linewidth=2, 
            label=f'决策边界 (x={decision_boundary:.2f})')
plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)

# 绘制训练数据点
plt.scatter(X[y==0], np.zeros(sum(y==0)), c='blue', marker='|', s=100, alpha=0.7)
plt.scatter(X[y==1], np.ones(sum(y==1)), c='green', marker='|', s=100, alpha=0.7)

plt.xlabel('花瓣宽度 (cm)', fontsize=12)
plt.ylabel('概率', fontsize=12)
plt.title('逻辑回归: Sigmoid 概率曲线', fontsize=14)
plt.legend(loc='center right')
plt.grid(True, alpha=0.3)
plt.ylim(-0.1, 1.1)
plt.show()

print("Sigmoid函数: σ(z) = 1 / (1 + e^(-z))")
print(f"决策边界: 花瓣宽度 = {decision_boundary:.2f} cm")

In [ ]:
# ============================================================
# 使用模型进行预测
# ============================================================

# 测试不同花瓣宽度的样本
test_samples = [[1.7], [1.5], [2.0], [1.0]]

print("预测结果:")
print("-" * 50)
print(f"{'花瓣宽度':<10} {'预测类别':<12} {'概率':<20}")
print("-" * 50)

for sample in test_samples:
    pred_class = log_reg.predict([sample])[0]
    pred_proba = log_reg.predict_proba([sample])[0]
    class_name = "Virginica" if pred_class == 1 else "非Virginica"
    print(f"{sample[0]:<10} {class_name:<12} P(V)={pred_proba[1]:.4f}")
    
print("-" * 50)

In [ ]:
# ============================================================
# 多分类：Softmax 回归
# ============================================================
# 
# 当类别数 > 2 时，使用 Softmax 回归
# Softmax 函数将得分转换为概率分布：
# P(y=k) = exp(z_k) / Σexp(z_j)
# 
# 确保所有类别概率之和为 1

# 使用两个特征：花瓣长度和花瓣宽度
X = iris['data'][:, (2, 3)]  # petal length, petal width
y = iris['target']  # 3个类别：0, 1, 2

print(f"特征形状: {X.shape}")
print(f"类别: {iris['target_names']}")

# 创建 Softmax 回归模型
# solver='lbfgs': 使用 L-BFGS 优化算法
# C=10: 正则化强度的倒数（C越大，正则化越弱）
softmax_reg = LogisticRegression(solver="lbfgs", C=10, random_state=42)

# 训练模型
softmax_reg.fit(X, y)

# 预测新样本
test_sample = [[5, 2]]  # 花瓣长度=5cm, 花瓣宽度=2cm
pred_class = softmax_reg.predict(test_sample)
pred_proba = softmax_reg.predict_proba(test_sample)

print(f"\n测试样本: 花瓣长度={test_sample[0][0]}cm, 花瓣宽度={test_sample[0][1]}cm")
print(f"预测类别: {iris['target_names'][pred_class[0]]}")
print(f"\n各类别概率:")
for i, name in enumerate(iris['target_names']):
    print(f"  {name}: {pred_proba[0, i]:.4f} ({pred_proba[0, i]*100:.1f}%)")

In [ ]:
# ============================================================
# 可视化 Softmax 回归决策边界
# ============================================================

# 创建网格点
x0_min, x0_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
x1_min, x1_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx0, xx1 = np.meshgrid(np.linspace(x0_min, x0_max, 100),
                        np.linspace(x1_min, x1_max, 100))

# 预测网格点的类别
X_grid = np.c_[xx0.ravel(), xx1.ravel()]
y_pred = softmax_reg.predict(X_grid).reshape(xx0.shape)

plt.figure(figsize=(10, 6))

# 绘制决策边界
plt.contourf(xx0, xx1, y_pred, alpha=0.3, cmap='RdYlGn')

# 绘制训练样本
colors = ['red', 'yellow', 'green']
for i, name in enumerate(iris['target_names']):
    mask = y == i
    plt.scatter(X[mask, 0], X[mask, 1], c=colors[i], 
                edgecolors='black', s=50, label=name)

plt.xlabel('花瓣长度 (cm)', fontsize=12)
plt.ylabel('花瓣宽度 (cm)', fontsize=12)
plt.title('Softmax 回归: 三类分类决策边界', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# ============================================================
# 知识总结
# ============================================================
print("\n" + "=" * 60)
print("逻辑回归总结")
print("=" * 60)
print("""
1. 二分类逻辑回归:
   - 使用 Sigmoid 函数: σ(z) = 1/(1+e^(-z))
   - 输出概率 P(y=1|x)
   - 决策边界: P = 0.5

2. 多分类 Softmax 回归:
   - 使用 Softmax 函数归一化概率
   - 输出每个类别的概率分布
   - 概率之和 = 1

3. 常用参数:
   - C: 正则化强度的倒数
   - penalty: 正则化类型 ('l1', 'l2')
   - solver: 优化算法 ('lbfgs', 'liblinear', 'saga')

4. 优点:
   - 输出概率，可解释性强
   - 训练速度快
   - 不易过拟合（有正则化）
""")